In [1]:
# LIST OF UPDATES
#
# Added bias (constant) to each linear layer
# Switched to using equivalent keras optimizer - SGD 
# Moved loss calculation to an independent function
# Switched to defining the network as Keras layers(!)
# Add tf.function to training loop - 10x speed increase(!)
# Switch to calculating gradients with keras 'trainable_weight' vs tf 'variables' 
# Tried using Keras Model.fit but order of magnitude slower than custom loop!
# Switch to train/val/test -> Showed that validation accuracy peaks at about 55% Training accuracy :(
# Switch to MeanSquaredError as loss function (But this is not a good error for classification)
# TODO : Early completion based on validation changes
# TODO : Represent data using windowing method

In [1]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

import numpy as np
import pandas as pd

import quantutils.dataset.pipeline as ppl
import quantutils.dataset.ml as mlutils
from marketinsights.api.model import MarketInsightsModel
from marketinsights.remote.ml import MIAssembly

2023-05-04 23:22:01.315545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-04 23:22:01.445719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-04 23:22:01.445740: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-04 23:22:02.661069: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
DATASET_ID1 = "4234f0f1b6fcc17f6458696a6cdf5101"  # DOW
NUM_FEATURES = (2 * 4) + 1

assembly = MIAssembly(secret="marketinsights-k8s-cred")

# Dataset
dataset, descriptor = assembly.get_dataset_by_id(DATASET_ID1, debug=False)
dataset = tf.cast(dataset, tf.float32)

# Split into Train/Val/Test
n = len(dataset)
train_df = dataset[0:int(n*0.7)]
val_df = dataset[int(n*0.7):int(n*0.9)]
test_df = dataset[int(n*0.9):]

# Split into features/labels
train_x, train_y = train_df[:, :NUM_FEATURES], train_df[:,NUM_FEATURES:]
val_x, val_y = val_df[:, :NUM_FEATURES], val_df[:,NUM_FEATURES:]
test_x, test_y = test_df[:, :NUM_FEATURES], test_df[:,NUM_FEATURES:]


2023-05-04 23:22:04.731450: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 23:22:04.732445: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-04 23:22:04.732517: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-04 23:22:04.732580: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-04 23:22:04.732643: W tensorflow/c

In [15]:
%%time
mlmodel = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(HIDDEN_UNITS, activation="sigmoid"),
        tf.keras.layers.Dense(NUM_LABELS),
    ]
)
mlmodel.compile(optimizer="SGD", loss="mse")

CPU times: user 22.3 ms, sys: 0 ns, total: 22.3 ms
Wall time: 20 ms


In [20]:
%%time

mlmodel.fit(train_x, train_y, batch_size=len(train_x), epochs=1000, verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='loss', patience=0)])

Epoch 1/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.2495
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 0.2495
CPU times: user 95.8 ms, sys: 3.37 ms, total: 99.2 ms
Wall time: 89 ms


In [3]:
NUM_LABELS = 1
HIDDEN_UNITS = 32

# Prepare our layer, loss, and optimizer.
mlmodel = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(HIDDEN_UNITS, activation="sigmoid"),
        tf.keras.layers.Dense(NUM_LABELS),
    ]
)

@tf.function
def loss_fn(model, y_pred, y, lam=tf.constant(0.001, tf.float32)):
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred))

    # Regularization using L2 Loss function
    regularizer = tf.nn.l2_loss(model.variables[0]) + tf.nn.l2_loss(model.variables[1])
    reg = (lam / tf.cast(tf.shape(y)[0], tf.float32)) * regularizer
    loss_reg = loss + reg

    return loss_reg

mse = tf.keras.losses.MeanSquaredError()

@tf.function  # Make it fast.
def train_on_batch(x, y):
    with tf.GradientTape() as tape:
        logits = mlmodel(x)
        #loss = loss_fn(mlmodel, logits, y)
        loss = mse(y, tf.nn.sigmoid(logits))
        gradients = tape.gradient(loss, mlmodel.trainable_weights)
    optimizer.apply_gradients(zip(gradients, mlmodel.trainable_weights))
    return loss


In [7]:
%%time

# Set training parameters
epochs = 10000
learning_rate = .1
losses = []

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# Format training loop
for epoch in range(epochs):
    epoch_loss = train_on_batch(train_x, train_y)
        
    # Keep track of model loss per epoch
    losses.append(epoch_loss)
    if epoch % 1000 == 0:
        print(f'Step {epoch} - Training Loss: {epoch_loss}, Val Loss: {mse(tf.nn.sigmoid(mlmodel(val_x)), val_y).numpy()}')
        #print(f'{epoch} - Accuracy: {accuracy.result():.2%}, Loss: {epoch_loss.numpy():0.3f}')
    


Step 0 - Training Loss: 0.2483554482460022, Val Loss: 0.24932557344436646
Step 1000 - Training Loss: 0.24834215641021729, Val Loss: 0.24927736818790436
Step 2000 - Training Loss: 0.24833020567893982, Val Loss: 0.24923017621040344
Step 3000 - Training Loss: 0.24831931293010712, Val Loss: 0.2491840124130249
Step 4000 - Training Loss: 0.24830929934978485, Val Loss: 0.24913868308067322
Step 5000 - Training Loss: 0.2483000010251999, Val Loss: 0.24909420311450958
Step 6000 - Training Loss: 0.24829129874706268, Val Loss: 0.24905049800872803
Step 7000 - Training Loss: 0.24828311800956726, Val Loss: 0.24900750815868378
Step 8000 - Training Loss: 0.24827536940574646, Val Loss: 0.2489652782678604
Step 9000 - Training Loss: 0.2482680231332779, Val Loss: 0.24892373383045197
CPU times: user 12.1 s, sys: 589 ms, total: 12.7 s
Wall time: 8.77 s


In [177]:
mlutils.evaluate(ppl.onehot(tf.nn.sigmoid(mlmodel(train_x)).numpy()), ppl.onehot(train_y.numpy()), threshold=0.5)

Won : 532.0
Lost : 431.0
Total : 963.0
Diff : 101.0
Edge : 10.488058151609554%
Information Coefficient : 0.10488057136535645


0.5524403

In [180]:
mlutils.evaluate(ppl.onehot(tf.nn.sigmoid(mlmodel(val_x)).numpy()), ppl.onehot(val_y.numpy()), threshold=0.5)

Won : 138.0
Lost : 138.0
Total : 276.0
Diff : 0.0
Edge : 0.0%
Information Coefficient : 0.0


0.5

In [179]:
mlutils.evaluate(ppl.onehot(tf.nn.sigmoid(mlmodel(test_x)).numpy()), ppl.onehot(test_y.numpy()), threshold=0.5)

Won : 73.0
Lost : 65.0
Total : 138.0
Diff : 8.0
Edge : 5.797101449275362%
Information Coefficient : 0.05797100067138672


0.5289855